In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
import librosa
import librosa.display
from scipy.signal import butter, filtfilt
from IPython.display import FileLink

In [ ]:
audio1, freq = librosa.load(r"C:\Users\taian\UFSC\LVA\teste1.waptt.wav", sr=None)

In [ ]:
Audio(data=audio1, rate=freq)

In [ ]:
#loudness

potencia = np.square(audio1)
loudness = np.mean(potencia)

def resultLoudness(loudness):

    if loudness > 0.05:
        return "Som barulhento."
    else:
        return "Som baixo."

print(resultLoudness(loudness))

"""como a audição humana está mais associada com a energia do som
do que com a amplitude, fiz o quadrado da amplitude e tirei a média,
a escolha do 0.05 foi com base nos valores comuns para sons 'neutros', 
já que sons comuns variam entre 0.001 e 0.2"""

In [ ]:
#sharpness

mediaEspectro = np.mean(np.abs(librosa.stft(audio1)), axis=1)
taxaAmostral = librosa.fft_frequencies(sr=freq)

mediaFreq = np.sum(taxaAmostral * mediaEspectro) / np.sum(mediaEspectro)

print(f"Frequência média: {mediaFreq:.2f} Hz")

def resultSharpness(mediaFreq):

    if mediaFreq < 250:
        return "Som grave."
    elif 250 < mediaFreq < 2500:
        return "Som médio."
    else:
        return "Som agudo."

print(resultSharpness(mediaFreq))

"""primeiro fiz a transformada para ter as frequencias de cada ponto e
depois tirei a média para verificar em qual frequencia o audio está concentrado"""

In [ ]:
#flutuação lenta
varLenta = librosa.feature.rms(y=audio1, frame_length=2048)[0]
fluctStrength = np.std(varLenta)

#flutuação rápida
roughness = np.mean(np.abs(np.diff(audio1)))

print(f"Fluctuation Strength: {fluctStrength:.6f}")
print(f"Roughness: {roughness:.6f}")

def resultStrenght(fluctStrength):

    if fluctStrength > 0.01:
        return "Possui flutuações lentas."
    else:
        return "O som é estável."

print(resultStrenght(fluctStrength))

def resultRoughness(roughness):
    if roughness > 0.02:
        return "Possui flutuação rápida."
    else:
        return "O som é suave."

print(resultRoughness(roughness))

"""'separei' o audio em janelas e depois tirei o desvio padrão (como a janela é
grande, percebe variações lentas de volume), quanto maior o desvio padrão, mais varia,
ou seja, indica que existe flutuação, por fim, verifiquei a diferença entre as janelas
(para verificar se existe variações rápidas)"""

In [ ]:
#tonality

mediaEspec = np.mean(np.abs(librosa.stft(audio1)), axis=1)
frequencias = librosa.fft_frequencies(sr=freq)
freqDominante = frequencias[np.argmax(mediaEspec)]

print(f"Frequência dominante: {freqDominante:.2f} Hz")

def resultTonality(freqDominante):

    if freqDominante < 300:
        return "Tonalidade grave." #(ronco, batida)
    elif freqDominante < 1500:
        return "Tonalidade média." #(voz humana)
    elif freqDominante < 5000:
        return "Tonalidade sibilante." #('s' ou 'sh')
    else:
        return "Tonalidade assobiante ou muito aguda." #(como apitos)

print(resultTonality(freqDominante))

"""a analise é bem parecida com a do sharpness, porém nesse caso
usei a frequencia que mais concentrada 'dominante' para estimar o tonality,
o que não deixa de ser uma classificação subjetiva"""

In [ ]:
#pitch

pitch, magnitudes = librosa.piptrack(y=audio1, sr=freq)

freqPitch = []
for i in range(pitch.shape[1]):
    idx = np.argmax(magnitudes[:, i])
    frequencia = pitch[idx, i]
    if frequencia > 0:
        freqPitch.append(frequencia)

plt.figure(figsize=(10, 4))
plt.plot(freqPitch)
plt.title("Variação do Pitch ao longo do tempo")
plt.ylabel("Frequência (Hz)")
plt.xlabel("Quadros")
plt.grid()
plt.show()

"""primeiro usei o piptrack para obter a frequencia fundamental de cada
janela e a intensidade de cada pitch, peguei as frequencias com maior
intensidade para estimar o pitch (já que tem maior 'probabilidade' de ser o pitch),
e por fim plotei no gráfico"""

In [ ]:
classLoudness = resultLoudness(loudness)
classSharpness = resultSharpness(mediaFreq)
classFluctuation = resultStrenght(fluctStrength)
classRoughness = resultRoughness(roughness)
classTonality = resultTonality(freqDominante)

resultados = {
    "Loudness": loudness,
    "Classificação Loudness": classLoudness,
    "Sharpness": mediaFreq,
    "Classificação Sharpness": classSharpness,
    "Fluctuation Strength": fluctStrength,
    "Classificação Flutuação": classFluctuation,
    "Roughness": roughness,
    "Classificação Roughness": classRoughness,
    "Tonality": freqDominante,
    "Classificação Tonality": classTonality,
    "Pitch": sum(freqPitch)/len(freqPitch) if freqPitch else 0
}

dfResults = pd.DataFrame([resultados])

dfResults.to_csv("relatorioAudio.csv", index=False)

FileLink(r'relatorio_audio.csv')
print("Relatório salvo em relatorioAudio.csv")
